In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')


import re
from konlpy.tag import Okt, Mecab
from ckonlpy.tag import Twitter
from hanspell import spell_checker
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('stopwords')

# -------
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

import pickle
with open( "data/add_user_dictionary.pickle", "rb" ) as file:
    add_dict = pickle.load(file)

twitter = Twitter()
# 특정 단어 추가
twitter.add_dictionary(add_dict, 'Noun')

m = Mecab(r'C:/mecab/mecab-ko-dic')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cv002\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 데이터 불러오기

In [121]:
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x).lower())
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,various artists
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,murray perahia
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,peter gabriel
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,matoma
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,jude law


In [16]:
train = pd.read_json('data/train.json')
train['tags__'] = train['tags'].apply(lambda x : " ".join(x))
train['plylst_title__'] = train['plylst_title']
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락,여행같은 음악
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상,요즘 너 말야
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한,"편하게, 잔잔하게 들을 수 있는 곡.-"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,크리스마스 분위기에 흠뻑 취하고 싶을때
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스,추억의 노래 ㅋ


In [8]:
test = pd.read_json('data/0918spell_check_train.json')
test.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락,여행 같은 음악
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상,요즘 너 말이야
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,카페 잔잔한,편하게 잔잔하게 들을 수 있는 곡
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈 오는 날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울 왕국 ...,크리스마스 분위기에 흠뻑 취하고 싶을 때
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스,추억의 노래


## 해당 문자가 포함되어 있는 열 찾아오기

#### 개요
>'백현'같은 경우는 '백현 (BAEKHYUN)' 이렇게 쳐야지 백현이 나온다   
하지만 나는 '백현'이라고 치면 '백현 (BAEKHYUN)'을 찾아오게 만들고 싶다   
하지만 이 방식은 '백현'이라는 문자가 포함되어 있다면 모두 찾아와서 '백현진' 이런 것도 찾아오게 된다

In [17]:
song_meta.query('artist_name == "백현 (baekhyun)"').head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
3934,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Psycho (Bonus Track),"[GN2500, GN0200]",[백현 (BAEKHYUN)],3934,백현 (baekhyun)
51681,"[GN2502, GN2501, GN0303, GN2504, GN0301]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Stay Up (Feat. Beenzino),"[GN2500, GN0300]",[백현 (BAEKHYUN)],51681,백현 (baekhyun)
125181,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Diamond,"[GN0400, GN2500]",[백현 (BAEKHYUN)],125181,백현 (baekhyun)
186196,"[GN2502, GN2501, GN0101, GN2505]",20170414,바래다줄게 (Take You Home) - SM STATION,10055218,[672859],바래다줄게 (Take You Home) (Inst.),"[GN2500, GN0100]",[백현 (BAEKHYUN)],186196,백현 (baekhyun)
312216,"[GN0105, GN1501, GN0101, GN1504]",20200225,낭만닥터 김사부 2 OST,10393897,[672859],너를 사랑하고 있어,"[GN1500, GN0100]",[백현 (BAEKHYUN)],312216,백현 (baekhyun)


#### 가수 찾아오기

In [7]:
from itertools import chain

In [8]:
# 해당 가수 찾아오기
from itertools import chain

def find_singer(names = []):
    
    names = names.split()
    
    idx = []
    for name in names:
        A = [x for x in song_meta['artist_name'] if name in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(song_meta[song_meta['artist_name'] == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return song_meta.iloc[idx_list,:]

In [9]:
find_singer('백현')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
157484,"[GN1502, GN1501]",20021210,철없는 아내와 파란만장한 남편 그리고 태권소녀 OST,25904,"[2268, 168436]",구멍난 그림자,[GN1500],"[한대수, 백현진]",157484,한대수 백현진
180023,"[GN0105, GN0101]",20170214,비가와,10038305,"[490981, 672859]",비가와 (Rain),[GN0100],"[소유 (SOYOU), 백현 (BAEKHYUN)]",180023,소유 (soyou) 백현 (baekhyun)
121303,"[GN0105, GN0101]",20160513,The Day - SM STATION,2684689,"[175139, 672859]",The Day,[GN0100],"[케이윌, 백현 (BAEKHYUN)]",121303,케이윌 백현 (baekhyun)
468075,[GN0101],20160513,The Day - SM STATION,2684689,"[175139, 672859]",The Day (Inst.),[GN0100],"[케이윌, 백현 (BAEKHYUN)]",468075,케이윌 백현 (baekhyun)
239509,[GN0701],20070312,The Unique,347021,[221002],그리운 그사람,[GN0700],[백현우],239509,백현우
...,...,...,...,...,...,...,...,...,...,...
556642,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20110218,찰라의 기초,1179153,[168436],어떤 냄새,"[GN0500, GN0600]",[백현진],556642,백현진
572676,[GN0801],20191129,가볍고 수많은,10358776,[168436],반 줄,[GN0800],[백현진],572676,백현진
576100,"[GN1502, GN1501]",20050408,주먹이 운다 OST,301617,[168436],행복의 나라로,[GN1500],[백현진],576100,백현진
637548,"[GN0508, GN0501, GN0601, GN0503, GN0605]",20080519,Time Of Reflection,381052,[168436],눈물 닦은 눈물,"[GN0500, GN0600]",[백현진],637548,백현진


#### 해당 제목 찾아오기

In [165]:
from itertools import chain

def find_title(titles):
    titles = titles.split()
    idx = []
    for title in titles:
        A = [x for x in train['plylst_title'] if title in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(train[train['plylst_title']  == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return train.iloc[idx_list,:]

In [187]:
find_title('K_POP') 

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__


#### 해당 태그 찾아오기

In [17]:
from itertools import chain

def find_tag(tags):
    tags = tags.split()
    idx = []
    for tag in tags:
        A = [x for x in train['tags__'] if tag in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(train[train['tags__']  == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return train.iloc[idx_list,:]

In [18]:
find_tag('강남') # kpop k_pop KPOP

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
83270,"[일렉트로니카, 홍대, 드라이브, EDM, 클럽, 강남, 일렉, 신나는]",62486,VOCAL EDM,"[339639, 621987, 82407, 36030, 389448, 107248,...",35,2019-09-08 13:42:30.000,일렉트로니카 홍대 드라이브 EDM 클럽 강남 일렉 신나는,VOCAL EDM
93370,"[힙합, 클럽, 홍대, 운동, 흑인, 서울대, 힙합클럽, 현피꼼장어, 이태원, 강남]",16339,350% BLACK HIP-HOP [힙합클럽이 그리운 분들께],"[275831, 416315, 214070, 371478, 658768, 39929...",5357,2020-04-16 19:23:43.000,힙합 클럽 홍대 운동 흑인 서울대 힙합클럽 현피꼼장어 이태원 강남,350% BLACK HIP-HOP [힙합클럽이 그리운 분들께]
13694,"[힙합, 클럽, 파티, 랩, Club, HipHop, 이태원, 강남, Rap, 홍대]",64321,오늘은 힙합 클럽으로,"[227812, 465555, 16083, 91025, 508043, 154933,...",10,2017-10-21 17:25:08.000,힙합 클럽 파티 랩 Club HipHop 이태원 강남 Rap 홍대,오늘은 힙합 클럽으로
38397,"[클럽, 홍대, 주말, 클럽음악, 드라이브, 불금, 기분전환, 강남, 일렉, 신나는]",120625,핫한 클럽에 나오는 핫한 클럽음악 플레이리스트!!,"[33104, 349571, 17760, 449712, 171710, 497283,...",18,2019-08-01 23:54:35.000,클럽 홍대 주말 클럽음악 드라이브 불금 기분전환 강남 일렉 신나는,핫한 클럽에 나오는 핫한 클럽음악 플레이리스트!!
37067,"[클럽, 운동, 드라이브, EDM, 강남, 스트레스, 일렉]",64396,"미친듯이 터지는 실패없는 EDM 모음 (클럽,일렉) - 입문용 2017","[23359, 345169, 425668, 61747, 256812, 66214, ...",2666,2018-10-08 01:29:46.000,클럽 운동 드라이브 EDM 강남 스트레스 일렉,"미친듯이 터지는 실패없는 EDM 모음 (클럽,일렉) - 입문용 2017"
6535,"[강남클럽, UMF, 이태원클럽]",45565,한번 들으면 다음곡이 궁금해지는 일렉트로닉,"[208878, 233443, 280576, 43627, 157601, 616270...",2,2017-09-11 16:35:39.000,강남클럽 UMF 이태원클럽,한번 들으면 다음곡이 궁금해지는 일렉트로닉
30778,"[힙합, 클럽, 홍대, nb, 힙합클럽, 2000, 강남, 탑골, 엔비, 앤비]",78947,2000년대 그 시절 그 감성_ 엔비(NB)에서 듣던 힙했던 곡들,"[56991, 705381, 221562, 148875, 465555, 431880...",374,2020-04-04 21:43:17.000,힙합 클럽 홍대 nb 힙합클럽 2000 강남 탑골 엔비 앤비,2000년대 그 시절 그 감성_ 엔비(NB)에서 듣던 힙했던 곡들
110356,"[힙합, 힙존, 강남클럽, 외국힙합, 이태원, 외힙, 아레나, 힙합클럽, 힙클, 아...",62483,진정한 힙스터,"[8587, 337113, 347823, 251940, 237089, 601369,...",1,2019-01-25 15:52:45.000,힙합 힙존 강남클럽 외국힙합 이태원 외힙 아레나 힙합클럽 힙클 아레나힙존,진정한 힙스터
88441,"[일렉트로니카, 홍대, EDM, 클럽, 강남, 하우스, 일렉]",66928,찐한 EDM으로 스트레스를 날려보자,"[566757, 35165, 454782, 488567, 632878, 114977...",18,2017-04-28 08:16:42.000,일렉트로니카 홍대 EDM 클럽 강남 하우스 일렉,찐한 EDM으로 스트레스를 날려보자
5388,"[앤써, 바운드, 매스, 디스타, 강남메이드, 써클, 헤일로, 아레나, 스티브, 옥타곤]",58852,(강남클럽)아레나&바운드&써클&디스타&에비뉴535&옥타곤&매스&강남메이드,"[559941, 40907, 663434, 188555, 639051, 660710...",218,2017-09-20 12:03:40.000,앤써 바운드 매스 디스타 강남메이드 써클 헤일로 아레나 스티브 옥타곤,(강남클럽)아레나&바운드&써클&디스타&에비뉴535&옥타곤&매스&강남메이드


---
---
---

## 사용자 사전 추가

### Mecab 사용자 사전 추가

- 가수 이름 추가 
 - artist_name을 다 찾아 오려고 했으나 ,가 붙어있으면 저장이 되지않는다
 - 그냥 특수문자를 제거한 단어들만 추가
 - 영어 같은 경우 알파벳이 추가되어서 길이가 2이하인 영어 글자는 다 지워줌
 - 숫자 같은 경우도 추가되서 숫자도 다 지워줬음
  
- 숫자 + 단위명사 (SN+NNBC)
    - **회, 년, 월, 일** 이 경우는 twitter에서 Number로 인식해서 문제가 되지않는다
    - 문제가 되는 경우는 **살, 대**같은 단위명사는 Number로 인식을 하지 못한다는 것이다.

    - 그래서 **10살, 10대**같이 Number로 인식을 하지 못하는 단어들은 twitter에서 임의로 단어 사전으로 추가를 해줘야한다

    - 그렇게 하면 10살 10대 10회 10년 10월 10일 같이 나올 것이다 이 단어들의 품사는 모두 Number

    - 이렇게 나온 결과물들을 수사(NR)로 묶어주고 싶은데... 수사는 위의 단어는 수사에 해당시키지 못 할듯
        - 단위명사(NNBC)도 의존명사라서 가능은 할 거 같은데 잘 모르겠음

    - ~~그냥 **SN+NNBC라는 새로운 품사 파일을 생성해서 추가**함~~
    - 하지만 새로운 파일에 옳바른 형식으로 넣으면 가능하니 파일 이름을 잘 구분하면 될 듯 하다
    
    
### Twitter 사용자 사전 추가

- 원하는 단어를 1회성으로 추가를 할 수 있다
- 그렇기 때문에 이 노트북 파일을 벗어나면 사용이 불가능하다
- 계속해서 추가해서 사용할 사용자 사전이기 때문에 pickle형태로 바꿔서 저장해준다

>**twitter.add_dictionary(text, '품사')** 형식으로 입력한다  
**text**의 경우 list형태로도 가능하고 단일 text도 가능하다   
**품사**의 경우 Noun, Josa, Verb, Modifier, Punctuation, Number, Alpha가 존재한다   

In [16]:
# 사용자 사전 불러오기
import pickle

with open(r"C:\mecab\user-dic\nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()
file_data =list(set(file_data))
print(len(file_data))

with open( "data/add_user_dictionary.pickle", "rb" ) as file:
    loaded_data = pickle.load(file)
print(len(loaded_data))

22751
22751


In [83]:
# loaded_data = list(set(np.concatenate([re.sub('[^A-Za-z가-힣0-9]'," ",text).split() for text in file_data])))
# loaded_data.remove('NNP')
# loaded_data.remove('T')
# print(len(loaded_data))

22751


In [15]:

# 사용자 사전 추가
x = ['기분전환']

for X in x:
    file_data.append(f'{X},,,,NNP,*,T,{X},*,*,*,*,*\n')
    loaded_data.append(X)

file_data = list(set(file_data))
loaded_data = list(set(loaded_data))
    
with open(r"C:\mecab\user-dic\nnp.csv", 'w', encoding='utf-8') as f: 
    for line in file_data: 
        f.write(line)
        
with open( "data/add_user_dictionary.pickle", "wb" ) as file:
    pickle.dump(loaded_data, file)        

print(len(file_data))
print(len(loaded_data))

22751
22751


---
---
---

## 자연어 처리 비교

### PyKoSpacing / Py-hanspell

- 내가 볼 때는 이거보다 py-hanspell 성능이 더 좋은 것 같음 
    - py-hanspell의 경우 띄어쓰기도 내가 원하는 방향으로 된다
    - 맞춤법 문제도 해결해 준다

#### pykospacing 다운로드 및 연습

In [31]:
from pykospacing import Spacing
spacing = Spacing()

AttributeError: module 'h5py' has no attribute 'File'

In [10]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent,'\n')

spacing = Spacing()

kospacing_sent = spacing(new_sent) 

print(sent,'\n')
print(kospacing_sent,'\n')

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다. 

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다. 

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다. 



#### py-hanspell 다운로드 및 연습

In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [6]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [12]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent,'\n')
print(kospacing_sent,'\n')

김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다. 

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다. 



### Twitter
Twitter 사용자 사전 추가하고 사용자 사전 추가하기

추가된게 저장되지 않고 다른 노트북을 열때도 계속 추가를 해줘야 하는듯

In [ ]:
# Twitter 사용자 사전 추가 다운로드
# $ git clone https://github.com/lovit/customized_konlpy.git

# $ pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter
tagger = Twitter()

tagger.add_dictionary(['파이콘', '엔엘피', '아이오아이', '너무너무너무'], 'Noun')
tagger.add_dictionary(['8살'], 'Noun',force=True)

tagger.pos('8월')

### 자연어 처리 과정 / 클래스 비교 

1. 특수기호 제거
2. Py-hanspell로 띄어쓰기 및 맞춤법 처리(붙여쓰기는 불가능)
3. Twitter를 통해 정규화 및 어간 추출 형태소 분리
4. Mecab으로 원하는 품사만 추출
5. 불용어 처리

In [22]:
text = '기리보이의노래는길이보인다 / 외않되 / 그래욬ㅋㅋㅋ'

# step 1. 띄어쓰기 및 맞춤법 체크
print('Py-hanspell : ',spell_checker.check(text).checked, '\n') # 띄어쓰기 + 맞춤법 해결 가능 
print('PyKoSpacing : ',spacing(text), '\n') # 띄어쓰기만 해결 가능 하지만 원하는 느낌이 아님
# 사용자 사전을 추가하는 방법을 모르겠음

# step 2. 정규화
# 정규화는 Okt랑 Twitter만 가능함 
# 그렇기 때문에 Kkma, Komoran, Hannanum Mecab은 이 과정에서는 사용 불가능 
print('Okt : ',okt.morphs(text, stem = True, norm = True), '\n') # 속도 느림 

print('Twitter : ',twitter.morphs(text, stem = True, norm = True), '\n') # Okt보다 속도 빠름 / 일회성으로 원하는 단어 추가 가능 

# step 3. 세부품사 변경
print('Mecab : ',m.pos(text), '\n') # 속도 가장 빠름

Py-hanspell :  기리보이의 노래는 길이 보인다 / 왜 안돼 / 그래욬ㅋㅋㅋ 



NameError: name 'spacing' is not defined

## 자연어처리 

### 자연어 처리 함수 만들기 및 테스트 실험

In [5]:
# 띄어쓰기를 먼저 진행하는 것이 좋아 보임
# 띄어쓰기 맞춤법 체크 / 특수문자 제거
def spell(text):
    text = re.sub('[^A-Za-z가-힣0-9]'," ",text)
    txt = spell_checker.check(text).checked.lower()
    
    return txt

In [2]:
# 정규화 형태소 분석
def konlpy_preprocessing(text, removes_stopwords = False, stop_words = []):
    # Twitter를 활용해서 
    # stem = True > 어간 추출
    # norm = True > 정규화 진행
    morph = twitter.morphs(text, norm = True)
        
    parts_of_speech = []
    # mecab으로 원하는 품사만 뽑아오기
    for mor in morph:
        for word, tag in m.pos(mor):
            if tag in ['NNP', 'NNG', 'VA', 'VV', 'SL', 'SN', 'XR', 'VA+ETM', 'VV+EC+VX+ETM']:
                parts_of_speech.append(word)
    
#     stopwords에 있는 단어 제거
    if removes_stopwords == True:
        parts_of_speech = [token for token in parts_of_speech if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        parts_of_speech = [w for w in parts_of_speech if not w in stops] 
    
    return parts_of_speech

In [3]:
# 통합
def total_konlpy_preprocessing(text, removes_stopwords = False, stop_words = []):
    text = re.sub('[^A-Za-z가-힣0-9]'," ",text)
    txt = spell_checker.check(text).checked.lower()

    morph = twitter.morphs(txt, norm = True)
        
    parts_of_speech = []
    # mecab으로 원하는 품사만 뽑아오기
    for mor in morph:
        for word, tag in m.pos(mor):
            if tag in ['NNP', 'NNG', 'VA', 'VV', 'SL', 'SN', 'XR', 'VA+ETM', 'VV+EC+VX+ETM','VV+ETN']:
                parts_of_speech.append(word)
    
#     stopwords에 있는 단어 제거
    if removes_stopwords == True:
        parts_of_speech = [token for token in parts_of_speech if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        parts_of_speech = [w for w in parts_of_speech if not w in stops] 
    
    return parts_of_speech

In [3]:
konlpy_preprocessing('기분전환', removes_stopwords = True, stop_words = stopword)

['기분전환']

In [4]:
text = '기분전환' 
txt = spell_checker.check(text).checked
display(txt)

morph = twitter.pos(txt, stem = False, norm = True)
display(morph)
display(m.pos(text))

display(total_konlpy_preprocessing(text, removes_stopwords = True, stop_words = stopword))

'기분전환'

[('기분전환', 'Noun')]

[('기분전환', 'NNG')]

NameError: name 'total_konlpy_preprocessing' is not defined

In [56]:
x = []
for i in test['tags__']:
    x.append(m.pos(i))
    

x

[[('락', 'NNG')],
 [('추억', 'NNG'), ('회상', 'NNG')],
 [('카페', 'NNP'), ('잔잔', 'XR'), ('한', 'XSA+ETM')],
 [('연말', 'NNG'),
  ('눈', 'NNG'),
  ('오', 'VV'),
  ('는', 'ETM'),
  ('날', 'NNG'),
  ('캐럴', 'NNG'),
  ('분위기', 'NNG'),
  ('따듯', 'XR'),
  ('한', 'XSA+ETM'),
  ('크리스마스', 'NNP'),
  ('캐럴', 'NNG'),
  ('겨울', 'NNG'),
  ('노래', 'NNG'),
  ('크리스마스', 'NNP'),
  ('겨울', 'NNP'),
  ('왕국', 'NNG'),
  ('크리스마스', 'NNP'),
  ('송', 'NNP')],
 [('댄스', 'NNG')],
 [('운동', 'NNG'),
  ('드라이브', 'NNG'),
  ('pop', 'SL'),
  ('트로피컬', 'NNP'),
  ('하우스', 'NNP'),
  ('힐링', 'NNP'),
  ('기분전환', 'NNP'),
  ('2017', 'SN'),
  ('팝', 'NNG'),
  ('트렌드', 'NNG'),
  ('일렉', 'NNP')],
 [('짝사랑', 'NNG'),
  ('취향', 'NNG'),
  ('저격', 'NNG'),
  ('슬픔', 'NNG'),
  ('고백', 'NNG'),
  ('사랑', 'NNG'),
  ('이별', 'NNP')],
 [('잔잔', 'XR'), ('한', 'XSA+ETM'), ('추억', 'NNG'), ('회상', 'NNG')],
 [('일렉트로니카', 'NNP'),
  ('포크', 'NNP'),
  ('메탈', 'NNG'),
  ('락', 'NNG'),
  ('댄스', 'NNG'),
  ('인디', 'NNG')],
 [('록', 'NNG'),
  ('metal', 'SL'),
  ('이일우', 'NNP'),
  ('m', 'SL'),
  ('에센셜', 'NN

###  태그 자연어 처리

In [9]:
tag = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in test['tags__']]
display(len(tag))
display(tag)

115071

[['락'],
 ['추억', '회상'],
 ['카페', '잔잔'],
 ['연말',
  '눈',
  '날',
  '캐럴',
  '분위기',
  '따듯',
  '크리스마스',
  '캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울',
  '왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동', '드라이브', 'pop', '트로피컬', '하우스', '힐링', '기분전환', '2017', '팝', '트렌드', '일렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스', '인디'],
 ['록', 'metal', '이일우', '에센셜', '메탈', 'rock', '락'],
 ['k', 'pop', '댄스', '걸그룹', '댄스', '스트레스', '해소'],
 ['새해', '여행', '프로필', '음악', '카카오', '톡', '기분전환', '소원', '프로필', '소망', '다짐', '카톡'],
 ['듣', '우울', '힐링'],
 ['힙합', '느낌', '밤', '새벽', 'rnb', '감각', '드라이브', '국내', '그루브'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', 'pop'],
 ['봄', '사랑'],
 ['비', '날', '날'],
 ['목록', '폐막식', '올림픽', '엑소'],
 ['조용', '히', '새벽', '감성', '고민', '맥주', '잔'],
 ['카페', '재즈', '잔잔'],
 ['ost'],
 ['댄스'],
 ['감성', '어쿠스틱', '잔잔', '새벽', '편안', '인디', '밤'],
 ['잔잔', '밤', '새벽'],
 ['힐링', '드라이브', '에너지', '인디', '여행'],
 ['듀엣', '취향', '저격', '피처', '사랑', '남녀'],
 ['카페', '사랑'],
 ['힙합', '신인', '국힙', '인디', '신나'],
 [

In [10]:
tag_df = pd.DataFrame()
tag_df['tagss'] = tag
tag_df

,tagss
0,[락]
1,"[추억, 회상]"
2,"[카페, 잔잔]"
3,"[연말, 눈, 날, 캐럴, 분위기, 따듯, 크리스마스, 캐럴, 겨울, 노래, 크리스..."
4,[댄스]
...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]"
115067,[일렉]
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]"
115069,"[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길]"


In [11]:
tag_df.to_json('data/0917_tag_konlpy.json', orient = 'records')

### 타이틀 자연어 처리

In [12]:
title_to_tag = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test['plylst_title__']]
display(len(title_to_tag))
title_to_tag

115071

[['여행', '음악'],
 ['요즘'],
 ['편하', '잔잔', '곡'],
 ['크리스마스', '분위기', '취하'],
 ['추억', '노래'],
 ['2017', 'pop', 'trend'],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
 ['멍청이', '맘', '도'],
 ['dancing', 'moon', 'light', '01'],
 ['록', '메탈', 'written', '이일우'],
 ['걸그룹', '쓰', '쏭'],
 ['노래', '의지', '불', '태우', '1일', '1', '다짐', 'st', '용', '프로필', '뮤직'],
 ['지친', '위로', '제목', '편지', '준비'],
 ['트렌디', '그루브', '힙합', '알앤비', 'music'],
 ['autumn', 'jazz'],
 ['게임', '듣', '음악'],
 ['노래'],
 ['축가', '듀엣'],
 ['비', '내리', '날', '이면', '도'],
 ['올림픽', '목록'],
 ['조용', '히', '맥주', '한잔', '새벽', '감성'],
 [],
 ['추억', '명화', '만나', '영화', 'ost', '모음'],
 ['기분', '여름날'],
 ['지친', '하루', '끝', '힐링', '필요', '추', '천하', '인기', '곡'],
 ['새벽', '찬', '바람', '침대', '위', '국외'],
 ['기분', '햇살', '숨겨진', '명곡'],
 ['사랑', '남녀', '듀엣'],
 ['듣', '소래'],
 ['닥', '힙', '추', '8월', 'playlist'],
 ['거슈윈', '스타워즈', '미국', '대표', '클래식'],
 ['겨울', '기', '다리'],
 ['아침', '상쾌', '해지', '빅밴드', '스윙스', '윙'],
 ['여행', '길', '음악', '고고씽'],
 ['믿', '듣', '하우스', '일렉트로니카', 'pop', '2'],
 ['프러포즈', '음악'],
 ['일본',

In [13]:
title_to_tag_df = pd.DataFrame()
title_to_tag_df['title_to_tag'] =  title_to_tag
title_to_tag_df

,title_to_tag
0,"[여행, 음악]"
1,[요즘]
2,"[편하, 잔잔, 곡]"
3,"[크리스마스, 분위기, 취하]"
4,"[추억, 노래]"
...,...
115066,"[metal, e, sm, 2]"
115067,"[빠른, 리스, 위, 최신, 인기, edm, 모음]"
115068,"[1, 눈물, 가리, 이야기]"
115069,"[퇴근, 버스, 하루, 마무리, pop]"


In [14]:
title_to_tag_df.to_json('data/0917_title_to_tag_konlpy.json', orient = 'records')

### twitter phrases로 나눠주기

In [91]:
AA = test['tags__'] + ' ' + test['plylst_title__']
AA

0                                                락 여행 같은 음악
1                                            추억 회상 요즘 너 말이야
2                              카페 잔잔한 편하게  잔잔하게 들을 수 있는 곡  
3         연말 눈 오는 날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울 왕국 ...
4                                               댄스 추억의 노래  
                                ...                        
115066        록 메탈 밴드 사운드 록 락 메탈 메탈 락 extreme metal e sm  2
115067                    일렉 빠른 리스너를 위한 따끈따끈한 최신 인기 edm 모음 
115068    담시 가족 눈물 그리움 주인공 나의 이야기 사랑 친구  1  눈물이 앞을 가리는 나...
115069    잔잔한 버스 퇴근 버스 pop 풍경 퇴근길 퇴근 버스에서 편히 들으면서 하루를 마무...
115070             노래 추천 팝송 추천 팝송 팝송 모음 favorite popsong   
Length: 115071, dtype: object

In [110]:
B = []
for i in AA:
    C = word_tokenize(i)
    C = [w for w in C if not w in stop_words]
    C = [w for w in C if not w in stopword]
    B.append(C)

In [117]:
C = pd.DataFrame()
C['word'] = B
C.to_json('data/0917_word.json', orient = 'records')